## Goals: Hyper Parameter Optimisation of *QRF* model

This notebook propose different methods of hyper parameter optimisation based on X-Validation :
* Random Search
* Genetic algorithm [Not yet included]

# 1. Data Import and Setup

Imports necessary libraries, sets up environment paths.

In [69]:
# Standard library imports
import os
import sys
import json

# Third-party imports
from functools import partial
import numpy as np
import pandas as pd
from quantile_forest import RandomForestQuantileRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectKBest, f_regression

# Append project root to sys.path for local imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

# Local application imports
from src.utils.model import get_station_stats, custom_log_likelihood
from src.utils.SpatioTemporalSplit import SpatioTemporalSplit
from src.utils.custom_models import SnowIndexComputeTransformer


Defines constants :
* INPUT_DIR must be the same as the one defined in *00 Preprocessing/Feature Engineering*.
* MODEL_DIR is the directory where the exploration models will be saved.

In [70]:
INPUT_DIR = "../../../../data/input/"
MODEL_DIR = "../../../../models/exploration/"

SEED = 42 
ALPHA = 0.1
NUMBER_OF_WEEKS=4 

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10

DATASET_SPEC = "_".join(DATASET_TRANSFORMS)

if "pca" in DATASET_TRANSFORMS:
    DATASET_SPEC += f"_pct_{PCA_THRESHOLD}"

if "clust_index" in DATASET_TRANSFORMS:
    DATASET_SPEC += f"_geocl_{N_CLUSTER}"

if "clust_hydro" in DATASET_TRANSFORMS:
    DATASET_SPEC += f"_hydcl_{N_CLUSTER}"

DATASET_SPEC = "dataset_custom_6"

# columns to drop : target at different horizon, station_code, and features removed from Feature Selection
TO_DROP = ["water_flow_week1", "station_code", "water_flow_week2", "water_flow_week3", "water_flow_week4"]

# 2. Data Loading
Load in the baseline datasets, create the directory to save models.

In [71]:
# load the dataset
ds_train = pd.read_csv(f"{INPUT_DIR}{DATASET_SPEC}.csv")
train_data = ds_train.copy()
train_data.reset_index(inplace=True)
train_data = train_data.loc[:, ~train_data.columns.duplicated()]
ds_train = ds_train.set_index("ObsDate")
cv_data = train_data.copy()

y_train = {}
for i in range(0, NUMBER_OF_WEEKS):
    y_train[i] = train_data[f"water_flow_week{i+1}"]

### 3. Model preparation

Compute station statistics (usefull for scalling)

Create a custom Pipeline to keep track of the station code

Initialisation of the log likelihood scorer

In [72]:
def inverted_log_likelihood(estimator, X, y_true, cv_data, station_stats, alpha=0.1):
    return -custom_log_likelihood(estimator, X, y_true, cv_data, station_stats, alpha=alpha)

Initialisation of the SpatioTemporal Splitter

In [73]:
cv = SpatioTemporalSplit(
    n_splits=10,
    date_col='ObsDate',
    station_col='station_code',
    temporal_frac=0.68,
    spatial_frac=0.68,
    random_state=42
)

Fill NaN to avoid kbestSelector error

In [74]:
cv_data["water_flow_evolve_slope"].fillna(cv_data["water_flow_evolve_slope"].mean(), inplace=True)

C:\Users\chapu\AppData\Local\Temp\ipykernel_35228\2609022489.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cv_data["water_flow_evolve_slope"].fillna(cv_data["water_flow_evolve_slope"].mean(), inplace=True)


### 4. Hyper parameter tuning

Define the hyperparameter distributions for random search, take care the parameters presented here are choosen so that the search is fast you need to explore wider parameters range.

#### a. Random Search

In [75]:
cols_to_drop = TO_DROP.copy()
cols_to_drop += ["ObsDate"]
predictor_cols = [col for col in cv_data.columns if col not in cols_to_drop]

preprocessor = ColumnTransformer(transformers=[
    ('select', 'drop', cols_to_drop),
    ('kbest_select', SelectKBest(score_func=f_regression), predictor_cols)
], remainder='drop')

In [86]:
import re
N_ITER=45

qrf = {}
final_config = {}
X_incremental = {}
predictions = {}

for i in range(NUMBER_OF_WEEKS):

    print(f"Fitting week {i}")
    station_stats = get_station_stats(
        y_train[i].to_numpy(),
        train_data["station_code"].to_numpy()
    )
    
    scorer = partial(inverted_log_likelihood,
                    cv_data=cv_data,
                    station_stats=station_stats,
                    alpha=ALPHA)
    
    param_distributions = {
        "preprocessor__kbest_select__k": np.arange(3, len(predictor_cols) - 1, 3),
        'model__n_estimators': [5, 15, 35, 55, 85, 100, 130, 150, 190],
        'model__max_depth': [2, 7, 13, 20, 30, 50, 70, 100, 120, 160],
        'model__min_samples_leaf': [9, 15, 25, 35, 50, 60, 80, 105],
        'model__min_samples_split': [9, 15, 25, 35, 50, 60, 80, 105, 125, 150],
        'model__max_features': ['sqrt', 'log2', 0.3, 0.7, None],
        'model__bootstrap': [True, False]
    }

    if i == 0:
        qrf_model= RandomForestQuantileRegressor()

        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', qrf_model)
        ])

        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_distributions,
            n_iter=N_ITER,            # Number of parameter settings sampled
            scoring=scorer,       # Use our custom scorer
            cv=cv,                # Our custom spatio-temporal splitter
            n_jobs=-1,             # Use all available cores
            verbose=3,
            error_score='raise'
        )

        random_search.fit(cv_data, y_train[i])

        best_selector = random_search.best_estimator_.named_steps["preprocessor"].named_transformers_["kbest_select"]
        selected_mask = best_selector.get_support()
        adjusted_data = cv_data.drop(columns=cols_to_drop)
        selected_features = adjusted_data.columns[selected_mask]

        X_incremental[i] = cv_data[selected_features]
        
        print(random_search.best_params_)

        model_params = {
            key.replace('model__', ''): value
            for key, value in random_search.best_params_.items()
            if key.startswith('model__')
        }

        qrf[i] = RandomForestQuantileRegressor(**model_params)
        qrf[i].fit(X_incremental[i], y_train[i])

        final_config[f"week_{i}_config"] = {}
        final_config[f"week_{i}_config"]["best_params"] = random_search.best_params_
        final_config[f"week_{i}_config"]["best_score"] = random_search.best_score_
        final_config[f"week_{i}_config"]["features_to_train"] = selected_features.tolist()

    else:
        # Use the previous week's predictions as features
        y_pred = qrf[i - 1].predict(X_incremental[i - 1], quantiles=[0.5])
        y_pred = np.reshape(y_pred, (-1, 1))
        
        train_data = cv_data.copy(deep=True)

        train_data[f"week_{i}_pred"] = y_pred
        
        cols_to_add = [
            col for col in X_incremental[i - 1].columns
            if re.match(r"week_\d+_pred", col) or re.match(r"week_\d+_\d+_slope", col)
        ]

        print(f"Adding columns: {cols_to_add}")

        train_data[cols_to_add] = X_incremental[i - 1][cols_to_add]

        if i == 1:
            train_data[f"week_{i-1}_{i}_slope"] = (
                    train_data[f"week_{i}_pred"] - train_data["water_flow_lag_1w"]
                ) / train_data["water_flow_lag_1w"].replace(0, np.nan)
        else:
            train_data[f"week_{i-1}_{i}_slope"] = (
                    train_data[f"week_{i}_pred"] - train_data[f"week_{i-1}_pred"]
                ) / train_data[f"week_{i-1}_pred"].replace(0, np.nan)

        qrf_model= RandomForestQuantileRegressor()

        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', qrf_model)
        ])

        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_distributions,
            n_iter=N_ITER  ,            # Number of parameter settings sampled
            scoring=scorer,       # Use our custom scorer
            cv=cv,                # Our custom spatio-temporal splitter
            n_jobs=-1,             # Use all available cores
            verbose=3,
            error_score='raise'
        )

        random_search.fit(train_data, y_train[i])

        best_selector = random_search.best_estimator_.named_steps["preprocessor"].named_transformers_["kbest_select"]
        selected_mask = best_selector.get_support()
        adjusted_data = cv_data.drop(columns=cols_to_drop)
        selected_features = adjusted_data.columns[selected_mask]

        train_features = selected_features.tolist() + [f"week_{i-1}_{i}_slope", f"week_{i}_pred"] + cols_to_add

        X_incremental[i] = train_data[train_features]
        
        print(random_search.best_params_)

        model_params = {
            key.replace('model__', ''): value
            for key, value in random_search.best_params_.items()
            if key.startswith('model__')
        }

        qrf[i] = RandomForestQuantileRegressor(**model_params)
        qrf[i].fit(X_incremental[i], y_train[i])

        final_config[f"week_{i}_config"] = {}
        final_config[f"week_{i}_config"]["best_params"] = random_search.best_params_
        final_config[f"week_{i}_config"]["best_score"] = random_search.best_score_
        final_config[f"week_{i}_config"]["features_to_train"] = train_features


Fitting week 0
Fitting 10 folds for each of 45 candidates, totalling 450 fits


c:\Users\chapu\E.Lucas\Perso\Github\WaterScarcity.Hackathon\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'preprocessor__kbest_select__k': np.int64(30), 'model__n_estimators': 130, 'model__min_samples_split': 15, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Fitting week 1
Adding columns: []
Fitting 10 folds for each of 45 candidates, totalling 450 fits
{'preprocessor__kbest_select__k': np.int64(36), 'model__n_estimators': 15, 'model__min_samples_split': 35, 'model__min_samples_leaf': 35, 'model__max_features': 0.7, 'model__max_depth': 13, 'model__bootstrap': False}
Fitting week 2
Adding columns: ['week_0_1_slope', 'week_1_pred']
Fitting 10 folds for each of 45 candidates, totalling 450 fits
{'preprocessor__kbest_select__k': np.int64(33), 'model__n_estimators': 15, 'model__min_samples_split': 35, 'model__min_samples_leaf': 50, 'model__max_features': None, 'model__max_depth': 120, 'model__bootstrap': True}
Fitting week 3
Adding columns: ['week_1_2_slope', 'week_2_pred', 'week_0_1_slope', 'week_1_pred']
Fitting 10 folds for eac

In [88]:
with open(f"chained_config.json", 'w') as json_file:
    json.dump(final_config, json_file, indent=4, default=str)

In [87]:
final_config

{'week_0_config': {'best_params': {'preprocessor__kbest_select__k': np.int64(30),
   'model__n_estimators': 130,
   'model__min_samples_split': 15,
   'model__min_samples_leaf': 25,
   'model__max_features': None,
   'model__max_depth': 50,
   'model__bootstrap': True},
  'best_score': np.float64(-1.5103194009661183),
  'features_to_train': ['index',
   'precipitations_lag_1w_pca_1',
   'precipitations_lag_1w_pca_2',
   'precipitations_pca_1',
   'precipitations_pca_2',
   'tempartures_lag_1w_pca_1',
   'tempartures_pca_1',
   'tempartures_pca_2',
   'soil_moisture_pca_1',
   'soil_moisture_pca_2',
   'soil_moisture_pca_3',
   'evaporation_lag_1w_pca_1',
   'evaporation_pca_1',
   'soil_composition_pca_1',
   'soil_composition_pca_4',
   'soil_composition_pca_6',
   'soil_composition_pca_7',
   'soil_composition_pca_9',
   'latitude',
   'longitude',
   'catchment',
   'altitude',
   'water_flow_lag_1w',
   'water_flow_lag_2w',
   'water_flow_ma_4w_lag_1w_gauss',
   'north_hemisphere',

In [ ]:
for i in range(NUMBER_OF_WEEKS):
    print(X_incremental[i])

       soil_composition_pca_1  soil_composition_pca_9  latitude  longitude  \
0                    0.954040                0.128098  0.932117   0.854251   
1                    0.954040                0.128098  0.932117   0.854251   
2                    0.954040                0.128098  0.932117   0.854251   
3                    0.954040                0.128098  0.932117   0.854251   
4                    0.954040                0.128098  0.932117   0.854251   
...                       ...                     ...       ...        ...   
27697               -2.798534               -1.132140  0.009376   0.051031   
27698               -2.798534               -1.132140  0.009376   0.051031   
27699               -2.798534               -1.132140  0.009376   0.051031   
27700               -2.798534               -1.132140  0.009376   0.051031   
27701               -2.798534               -1.132140  0.009376   0.051031   

       catchment  water_flow_lag_1w  water_flow_lag_2w  \
0    